# Preliminaries

## Authentication

### Get temporary access token

In order to get an access token, we will send a POST request to /api/v1/auth/jwt/token endpoint with our username and password and grant_type=password as form fields in the body.

(If you get a 401 Unauthorized response for the sections below because the access token is expired, you can renew it by re-running the cell below)

In [ ]:
import ee
import requests
from authlib.integrations.requests_client import OAuth2Session
from authlib.oauth2.rfc7523 import ClientSecretJWT

username = "ksa@climateengine.com"
password = ""

body = {"grant_type": "password", "username": username, "password": password}
# note that we are using data= parameter instead of json=, because this endpoint expects x-www-form-urlencoded body
response = requests.post(
    "https://auth.climateengine.com/api/v1/auth/jwt/token", data=body
)
response_data = response.json()
print(f"status: {response.status_code}")
print(response_data)

access_token = response_data["access_token"]

### Test authentication by accessing a protected endpoint

In [ ]:
headers = {"Authorization": "Bearer " + access_token}
response = requests.get(
    "https://auth.climateengine.com/api/v1/users/me", headers=headers
)
print(f"status: {response.status_code}")
print(response.json())

## Create an OAuth2 Client and use it to obtain access tokens silently with authlib and requests

In order to create a client, we do an empty POST request to /api/v1/clients endpoint. client_secret is only shown during client creation and can't be accessed later, so make sure to save it somewhere safe.

In [ ]:
response = requests.post(
    "https://auth.climateengine.com/api/v1/clients/", headers=headers
)
response_data = response.json()
print(f"status: {response.status_code}")
print(response_data)

client_id = response_data["client_id"]
client_secret = response_data["client_secret"]

### Create an authlib client

The above client_id and client_secret can be used with authlib. The authlib client will automatically refresh the access tokens if they expire.

Note that authlib is just one choice of Python HTTP client.

In [ ]:
token_endpoint = "https://auth.climateengine.com/api/v1/auth/jwt/token"


def client_secret_jwt_client():
    client = OAuth2Session(
        client_id,
        client_secret,
        token_endpoint=token_endpoint,
        grant_type="client_credentials",
        token_endpoint_auth_method=ClientSecretJWT(token_endpoint),
    )
    client.fetch_token()
    return client


client = client_secret_jwt_client()
print("Getting an access_token with client_secret_jwt...")
print(client.fetch_token())

## Raster exports

In [ ]:
# ee.Authenticate()
ee.Initialize(project="my-image-assets-342021")

In [ ]:
# Define remaining parameters for API call and store as dictionary
dataset = "LANDSAT_SR"
variable = "NDVI"
temporal_statistic = "median"
start_date = "2020-11-01"
end_date = "2021-04-01"
bounding_box = [26.0820, 43.1627, 26.5862, 43.7892]

# No need to include an extension on the export_filename below, .tif will be appended automatically.
bucket_name = "ksa-sample-bucket"
export_filename = "landsat_ndvi_20201101_20210401_median"

# Set up parameters dictionary for API call
params = {
    "dataset": dataset,
    "variable": variable,
    "temporal_statistic": temporal_statistic,
    "start_date": start_date,
    "end_date": end_date,
    "bounding_box": f"{bounding_box}",
    "export_path": f"{bucket_name}/{export_filename}",
}

In [ ]:
response = client.get(
    "https://api.climateengine.net/v1/raster/export/values", params=params
)
export_response = response.json()
print(export_response)

In [ ]:
task_id = export_response["task_response"]["id"]
print(f"Processing task ID: {task_id}...")
print(ee.data.getTaskStatus(task_id))